In [1]:
# hide all code by default via JavaScript

%gui qt

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
#########################
# Imports
#########################

import os
import re
import napari
import numpy as np
import pandas as pd
from glob import glob
from skimage.io import imread
from napari.viewer import Viewer

#########################
# Hotkeys
#########################

def set_hotkeys():
    @Viewer.bind_key('F', overwrite=True)
    def nxtimg(viewer):
        """Next image."""
        next_image()
            
    @Viewer.bind_key('1', overwrite=True)
    def moveto1(viewer):
        """Move selected object to layer 1."""
        origin = viewer.active_layer
        destination = viewer.layers[1]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])
            origin.remove_selected()

    @Viewer.bind_key('2', overwrite=True)
    def moveto2(viewer):
        """Move selected object to layer 2."""
        origin = viewer.active_layer
        destination = viewer.layers[2]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])
            origin.remove_selected()


    @Viewer.bind_key('3', overwrite=True)
    def moveto3(viewer):
        """Move selected object to layer 3."""
        origin = viewer.active_layer
        destination = viewer.layers[3]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])
            origin.remove_selected()


    @Viewer.bind_key('4', overwrite=True)
    def moveto4(viewer):
        """Move selected object to layer 4."""
        origin = viewer.active_layer
        destination = viewer.layers[4]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])
            origin.remove_selected()

    @Viewer.bind_key('5', overwrite=True)
    def moveto5(viewer):
        """Move selected object to layer 5."""
        origin = viewer.active_layer
        destination = viewer.layers[5]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])  
            origin.remove_selected()

    @Viewer.bind_key('6', overwrite=True)
    def moveto6(viewer):
        """Move selected object to layer 6."""
        origin = viewer.active_layer
        destination = viewer.layers[6]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])          
            origin.remove_selected()
            
    @Viewer.bind_key('7', overwrite=True)
    def moveto7(viewer):
        """Move selected object to layer 7."""
        origin = viewer.active_layer
        destination = viewer.layers[7]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])          
            origin.remove_selected()
            
    @Viewer.bind_key('8', overwrite=True)
    def moveto8(viewer):
        """Move selected object to layer 8."""
        origin = viewer.active_layer
        destination = viewer.layers[8]
        data_idx = origin.selected_data

        for idx in data_idx:
            destination.add(origin.data[idx])          
            origin.remove_selected()


def get_imglist(btn):
    global imglist
    imglist = glob(os.path.join(folder.value, '*.jpg')) + \
                        glob(os.path.join(folder.value, '*.tif')) + \
                        glob(os.path.join(folder.value, '*.png'))
    
    imglist = sorted(imglist, key=lambda f: [int(n) for n in re.findall(r"\d+", f)])
    
    print(imglist)
    

def next_image(btn=None):
    global loaded
    global counter
    global imglist
    global skip
    
    if loaded:
        #for n in range(skip):
        if counter < len(imglist):
            save_labels()
            counter += 1
        
        viewer.layers.select_all()
        viewer.layers.remove_selected()
        viewer.reset_view()
    
    if counter < len(imglist):
        label_image(folder.value)
        loaded = True
        
def save_labels():
    global counter
    global imglist
    global namelist
    
    dic = {'category_id':[], 'x1':[], 'y1':[], 'x2':[], 'y2':[]}
    
    for n, name in enumerate(namelist):
        data = viewer.layers[name].data
        
        for sample in data:
            dic['category_id'].append(n)
            dic['x1'].append(sample[0][1])
            dic['y1'].append(sample[0][0])
            dic['x2'].append(sample[2][1])
            dic['y2'].append(sample[2][0])
            
        df = pd.DataFrame(dic)
        df.to_csv(os.path.splitext(imglist[counter])[0] + '_corrected.csv')
        
def label_image(path):  
    global viewer
    global counter
    global skip
    global namelist
    global colorlist
            
    image = imread(imglist[counter])
    
    inskip = counter % skip
    if inskip == 0:
        targets = pd.read_csv(os.path.splitext(imglist[counter])[0] + '_predict.csv')
    else:
        targets = pd.read_csv(os.path.splitext(imglist[counter - inskip])[0] + '_corrected.csv')
    
    viewer.add_image(image)
    
    boxes = []
    labels = []
    for row in targets.itertuples():
        boxes.append([row.x1, row.y1, row.x2, row.y2])
        labels.append(row.category_id)

    real_boxes = []
    class_n = 8

    for n in range(class_n):
        real_boxes.append([])

    for n,box in enumerate(boxes):
        real_boxes[labels[n]].append([[box[1], box[0]], [box[3], box[0]], [box[3], box[2]], [box[1], box[2]]])
        
    for n in range(class_n):
        viewer.add_shapes(real_boxes[n], shape_type='rectangle', edge_width=5,
                              edge_color=colorlist[n], face_color=colorlist[n], opacity=0.3, name=namelist[n], visible = not bool(n))

    viewer.active_layer = viewer.layers[-1]
    
    return 



#########################
# GUI
#########################

skip = 5
counter = 0
loaded = False
imglist = []
namelist = ['G1 (1)', 'G2 (2)', 'midS (3)', 'earlyS (4)', 'uncategorized (5)', 'lateS (6)', 'multinuc (7)', 'mito (8)']
colorlist = ['#FF0011', '#0000FF', '#FFB60B', '#45B65B', '#FFFFFF', '#F7F713', '#D0CC96', '#F621D3']

style = {'description_width': 'initial'}

from ipywidgets import Layout, HBox, VBox, Box, Checkbox, Text, BoundedFloatText, Button, FloatProgress, Label, Dropdown, Tab, Layout
from IPython.display import display

# Tab 1: basic settings

folder = Text(description='Image folder', style=style)

init = Button(description='Load Images!')
init.on_click(get_imglist)

go = Button(description='Next Image!')
go.on_click(next_image)

tab1 = VBox([folder, init, go])

display(tab1)


imglist = []
viewer = napari.Viewer()
set_hotkeys()